In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
processed = Path('../../data/processed')
selection_data = pd.read_pickle(Path(processed) / 'selection_data.pkl')

In [ ]:
selection_data.ChooseTeam.value_counts()

In [ ]:
selection_data = selection_data[selection_data.year >= 2000]
# selection_data = selection_data[(selection_data.year < 2000) & (selection_data.year >= 1960)]

In [ ]:
selection_data.b_PA_in_G.value_counts().sort_index()

In [ ]:
selection_data[['spot', 'home']].value_counts()

In [ ]:
def adjust_plot(plt):
    plt.xaxis.grid(False)
    plt.yaxis.grid(True, linestyle='dashed', color='lightgrey')
    plt.set_axisbelow(True)
    plt.xaxis.set_ticks_position('none') 
    plt.yaxis.set_ticks_position('none') 

    plt.spines['right'].set_visible(False)
    plt.spines['top'].set_visible(False)

In [ ]:
Wins = selection_data.loc[selection_data['Win'] == True, 'Win'].count()
Loses = selection_data.loc[selection_data['Win'] == False, 'Win'].count()
WinRate = round( (Wins / (Wins + Loses)) *100, 1)
print('Win pick rate: ' + str(WinRate) + '%')

In [ ]:
day_wins = selection_data.groupby('date')['Win'].prod()
print('Win both picks rate: ' + str(round(day_wins.mean()*100, 1)) + '%')

In [ ]:
def streak_counter(df):
    df['L_Win'] = df['Win'].shift(1)
    df['L_year'] = df['year'].shift(1)
    df['start_of_streak'] = (df['Win'] != df['L_Win']) | (df['year'] != df['L_year'])
    df['streak_id'] = df['start_of_streak'].cumsum()
    df['streak_counter'] = df.groupby('streak_id').cumcount() + 1
    df.loc[df.Win == 0, 'streak_counter'] = 0
    
    return df

In [ ]:
selection_data = selection_data.astype({'Win':'int8'})
selection_data = streak_counter(selection_data)
streaks = selection_data.groupby('streak_id').agg({'streak_counter':'max'})
streaks['streak_counter'].max()

In [ ]:
year_results = selection_data.groupby('year').agg({'Win':'mean', 'streak_counter':'max'})
year_results

In [ ]:
selection_data['EstProb'].describe()

In [ ]:
selection_data['bin'] = pd.qcut(selection_data['EstProb'], 10, duplicates='drop')

dots = selection_data.groupby(['bin'])[['EstProb', 'Win']].agg('mean')
dots = dots.reset_index()
dots.plot.scatter('EstProb','Win')

In [ ]:
selection_data[['EstProb', 'Win']].mean()

In [ ]:
# Start including data from current season

# Relief pitchers data

# Set up S3 if necessary for size, or for GPUs

# Check # training observations for each model fit 
# and make sure we aren't losing any

# Ensemble methods for NN and logistic

# Rest days

# Adjust Pitcher and Batter stats for park record ( merge it in and divide) - compare adjusted at home and away stats

# New pitcher flag?

# Find out where they are in the rotation

# Fix training so that it doesn't need to specify number of variables

# Split up grid search vs. non-grid search training

# Why is Win missing? Check merge loss

# Time-series CV

# Come up with potential guesses remaining counter (get schedule from retrosheet)

# Predict closer from roster and then use their rgs to construct estimated closing rgs (or just rgs for closing team)

# predicting game length

# Calculate streaks by aggregating predictions by day and not counting if any pick loses.

# add league dummy to model

# Average multiple prediction methods

# Come up with pitcher vs. Batter metrics

# check out distribution of PAs for selections. See if we can find a way to cut off the left tail.

# Park Factors by handedness

# Park Factors by hitter distribution (L vs. R, 1 v 2 v 3 v HR)

# Come up with Batter vs. Pitcher Stats and graph distribution to see tails.

# Add temperature and froce of win from or to

In [ ]:
restricted = selection_data.loc[
#     (selection_data['EstProb'] >= .74) &
#     (selection_data['park_factor'] >= 1) &
    (selection_data['spot'] <= 4),
    ['Win', 'year']
]

In [ ]:
restricted= restricted.astype({'Win':'int8'})
restricted = streak_counter(restricted)
streaks = restricted.groupby('streak_id').agg({'streak_counter':'max'})
streaks['streak_counter'].max()

In [ ]:
Wins = restricted.loc[
    (restricted['Win'] == True),
    'Win'].count()
Loses = restricted.loc[
    (restricted['Win'] == False),
    'Win'].count()
WinRate = round( (Wins / (Wins + Loses)) *100, 1)
print(str(WinRate) + '%')

In [ ]:
year_results = restricted.groupby('year').agg({'Win':'mean', 'streak_counter':'max'})
year_results

In [ ]:
year_results['streak_counter'].sum()

In [ ]:
selection_data = selection_data.sort_values(['year', 'Date'])

In [ ]:
selection_data['pick_day'] = selection_data.groupby(['year']).cumcount() + 1
selection_data['pick_day'] = np.ceil(selection_data['pick_day'] / 2)

In [ ]:
year_results.sort_values(['Win'], ascending=[False])